In [2]:
import ipywidgets as widgets
from IPython.display import display
dropdown = widgets.Dropdown(options=['A', 'B', 'C'], description='Choose:')
display(dropdown)

Dropdown(description='Choose:', options=('A', 'B', 'C'), value='A')

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install evaluate>=0.30
!pip install jiwer
!pip install accelerate -U
!pip install transformers[torch]

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-runtime-cu12 12.5.82 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
json_data['RecordingMetadata']['prompt']
json_data['SpeakerMetadata']['topik_level']

In [ ]:
# 1. 오디오 파일 경로 취합
import glob

path = "/content/drive/MyDrive/STTDATA/데이터모음/원본데이터/*"
raw_data_list = glob.glob(path)
raw_data_list = sorted(raw_data_list)
print(f"file_list : {raw_data_list[:10]}")
print(len(raw_data_list))

In [ ]:
# 2. json 파일 경로 취합
import glob

path = "/content/drive/MyDrive/STTDATA/데이터모음/정답데이터/*"
labeled_data_list = glob.glob(path)
labeled_data_list = sorted(labeled_data_list)
print(f"file_list : {labeled_data_list[:10]}")
print(len(labeled_data_list))

In [ ]:
from tqdm import tqdm
import json
import pandas as pd

transcript_list = []
for labeled_data in tqdm(labeled_data_list):
    with open(labeled_data, "r", encoding="utf-8") as f:
        json_data = json.load(f)
        json_data = json_data['RecordingMetadata']['orthographic']
        transcript_list.append(json_data)

df = pd.DataFrame(data=transcript_list, columns = ["transcript"])

# 텍스트 데이터로 만든 데이터프레임에 음성 파일 경로 컬럼을 추가
df["raw_data"] = raw_data_list

# Null data 유무 확인
df.isnull().values.sum()

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from datasets import Dataset, DatasetDict
from datasets import Audio

# 오디오 파일 경로를 dict의 "audio" 키의 value로 넣고 이를 데이터셋으로 변환
# 이때, Whisper가 요구하는 사양대로 Sampling rate는 16,000으로 설정한다.
ds = Dataset.from_dict({"audio": [path for path in df["raw_data"]],
                       "transcripts": [transcript for transcript in df["transcript"]]}).cast_column("audio", Audio(sampling_rate=16000))

# 데이터셋을 훈련 데이터와 테스트 데이터, 밸리데이션 데이터로 분할
train_testvalid = ds.train_test_split(test_size=0.2)
test_valid = train_testvalid["test"].train_test_split(test_size=0.5)
datasets = DatasetDict({
    "train": train_testvalid["train"],
    "test": test_valid["test"],
    "valid": test_valid["train"]})

# 작성한 데이터셋을 허깅페이스에 업로드
datasets.push_to_hub("jwh1449/AIhub_foreign_dataset")
print("완료")

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# 1. 오디오 파일 경로 취합
import glob

path = "/content/drive/MyDrive/STTDATA/데이터모음/원본데이터/*"
raw_data_list = glob.glob(path)
raw_data_list = sorted(raw_data_list)
print(f"file_list : {raw_data_list[:10]}")
print(len(raw_data_list))

In [ ]:
# 2. json 파일 경로 취합
import glob

path = "/content/drive/MyDrive/STTDATA/데이터모음/정답데이터/*"
labeled_data_list = glob.glob(path)
labeled_data_list = sorted(labeled_data_list)
print(f"file_list : {labeled_data_list[:10]}")
print(len(labeled_data_list))

In [ ]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer

# 파인튜닝을 진행하고자 하는 모델의 feature extractor를 로드
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")

# 파인튜닝을 진행하고자 하는 모델의 tokenizer를 로드
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="Korean", task="transcribe")

In [ ]:
from datasets import load_dataset

# 전체 데이터셋에서 각 split의 50%만 불러오기
low_call_voices = load_dataset(
    "jwh1449/AIhub_foreign_dataset",
    split={
        "train": "train[:50%]",
        "test": "test[:50%]",
        "valid": "valid[:50%]"
    }
)

low_call_voices.push_to_hub("jwh1449/AIhub_foreign_dataset2")
print("완료")

In [ ]:
from datasets import load_dataset

# 전체 데이터셋에서 각 split의 50%만 불러오기
low_call_voices = load_dataset(
    "jwh1449/AIhub_foreign_dataset",
    split={
        "train": "train[-50%:]",
        "test": "test[-50%:]",
        "valid": "valid[-50%:]"
    }
)

low_call_voices.push_to_hub("jwh1449/AIhub_foreign_dataset3")
print("완료")

In [ ]:
from datasets import load_dataset

# 데이터셋을 업로드할 때 접근을 제한하거나 비공개로 설정한 경우 허깅페이스 로그인이 필요하다.
low_call_voices = load_dataset("jwh1449/AIhub_foreign_dataset3")

def prepare_dataset(batch):
    # 오디오 파일을 16kHz로 로드
    audio = batch["audio"]

    # input audio array로부터 log-Mel spectrogram 변환
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # target text를 label ids로 변환
    batch["labels"] = tokenizer(batch["transcripts"]).input_ids
    return batch

# # 널값(결측치) 제거 함수 정의
# def remove_null(example):
#     return example["audio"] is not None and example["transcripts"] is not None

# # 널값 제거
# low_call_voices = low_call_voices.filter(remove_null)

low_call_voices = low_call_voices.map(prepare_dataset, remove_columns=low_call_voices.column_names["train"], num_proc=None)

# 전처리 작업이 오래 걸릴 수 있으므로, colab을 사용하여 파인튜닝을 진행한다면 전처리가 완료된 데이터셋을 Hub에 저장하는 것을 추천한다.
low_call_voices.push_to_hub("jwh1449/AIhub_foreign_dataset5")

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [16]:
import requests
import hashlib
import os
from datasets import load_dataset, DatasetDict

# # dataset_info.json의 download_checksums (실제로는 전체 목록 필요)
# # 아래는 예시로, 실제 dataset_info.json 파일에서 모든 URL과 체크섬을 가져와야 함
# dataset_info = {
#     "download_checksums": {"https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00000-of-00195.parquet": {"num_bytes": 100252687, "checksum": "1f32d12bb6db19b0cfdca6487858cce73c0653e551792dabbdaa8d91fa347b63"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00001-of-00195.parquet": {"num_bytes": 102863477, "checksum": "0c358a982ca204420635f39c14703a7d8de60c30a4da278485b8b324c0ae7e50"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00002-of-00195.parquet": {"num_bytes": 97553573, "checksum": "7422b478682fed3d911db92e53f5f5e97ba7de22e309e0df869c7e8c1d89b884"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00003-of-00195.parquet": {"num_bytes": 102058998, "checksum": "796d324dca3219b894431185a247a9b6e9ad6271ff33babd7b47526c5c2301e0"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00004-of-00195.parquet": {"num_bytes": 102557342, "checksum": "fcd2368818cc6b0aeb4f17483b41c855d45ee707f404cc3c5e3603b338f2a242"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00005-of-00195.parquet": {"num_bytes": 94543367, "checksum": "a68dd19bf94a0b760f6df058aced07849e34fb295106b6c401167d6a627f03bf"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00006-of-00195.parquet": {"num_bytes": 97549740, "checksum": "77c7d93c553049b7a0b94f2a1d4ec8c34e6d79fe9bb6f00e263c96e56e4ea92c"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00007-of-00195.parquet": {"num_bytes": 100679315, "checksum": "163e860d488cc8d4c6439ddb020bb8cc302b582690ecc4b455a4b376e99b385e"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00008-of-00195.parquet": {"num_bytes": 101449800, "checksum": "0bcdd9a596f7db111f44a98430b728975cd03a248b3ec4fec58963932fa7bba6"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00009-of-00195.parquet": {"num_bytes": 98479253, "checksum": "a6efacb41b5a5d28bc07ce4836d5066be691e0ef044e6081ecf210f778de9e01"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00010-of-00195.parquet": {"num_bytes": 98530080, "checksum": "97b5c591ddcb19ff703cd03c1aaa4eb17e1fa4ff1840897899377eaf3000fa8e"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00011-of-00195.parquet": {"num_bytes": 103423772, "checksum": "28ec08c1cb66cce83bd7171d274c2e3d0cd92230b9a86544d638c88094513279"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00012-of-00195.parquet": {"num_bytes": 100100765, "checksum": "e4a1bcf0b298c43b889f63e9fdbbf3b00af16e4e681c8626ed9ced19306eae46"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00013-of-00195.parquet": {"num_bytes": 97773711, "checksum": "b2d2a2c5b84f61f6838012fce5d2276497caa46ef110b3210c5c629df3f3384a"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00014-of-00195.parquet": {"num_bytes": 96435404, "checksum": "2d4257fdc13b21ab13074c68df677b3727be084677756aa5400511c85dd1f3a8"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00015-of-00195.parquet": {"num_bytes": 93360724, "checksum": "6e38451d11a428a4f35a59615d9dba4e93fcc2c16dc64b5bfc680f6113fb892f"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00016-of-00195.parquet": {"num_bytes": 102500345, "checksum": "6c43f756bebe5df5f2ca781552d1e85f3fcf64ef28ebb9315c049688a4babe9c"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00017-of-00195.parquet": {"num_bytes": 98567065, "checksum": "a374295dfb1f2e5030b6a008375cca4c3d1e772f6ab1e7330e87f7c492c8b92a"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00018-of-00195.parquet": {"num_bytes": 102443838, "checksum": "df57a8ec9f41c8835349e0e0765baf72b9bea3c8e2103b53a090a0fa183ba66a"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00019-of-00195.parquet": {"num_bytes": 99915229, "checksum": "c6e8b3c6e5b3637f69ad3616dd51fff20783dcc3fccd7b67ec3dfa7bdcce708e"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00020-of-00195.parquet": {"num_bytes": 98568022, "checksum": "e2c7758cd42bc6809856c39dc2ca9b69a28a6ec59eeccf0137d7090168bdc459"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00021-of-00195.parquet": {"num_bytes": 97740081, "checksum": "e1102f3242789683631f6643d83131b0c58733fbcadea7967817c871fb573ded"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00022-of-00195.parquet": {"num_bytes": 98912387, "checksum": "17ac8d8afbca377df97d8d2435b28f83a51f413d667ad3ad23a590f537c0bfb8"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00023-of-00195.parquet": {"num_bytes": 106989107, "checksum": "830ce6af6ccb668ab43764f4ea18e109328501ad28789064b4240d40b60819d5"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00024-of-00195.parquet": {"num_bytes": 99768537, "checksum": "50c13920d751d35b6ad9d578a9d92beed7f5d25a94b5904dd57ab69181f18d31"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00025-of-00195.parquet": {"num_bytes": 98668265, "checksum": "19b2b64abbff4442322359d45085602ebd92a974f1abbe32c469f3a9eab17c3e"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00026-of-00195.parquet": {"num_bytes": 101951922, "checksum": "bd32bb81793e5c41d9d5be7324e5eaa3f573edd9f1a4aa610b2f501d7699f267"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00027-of-00195.parquet": {"num_bytes": 104000646, "checksum": "990a7f359846a95f37a4017dea70bacd585cf663e28a6648d31275014d3e735a"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00028-of-00195.parquet": {"num_bytes": 98862855, "checksum": "2a5453a82c703aaf150c3ba943185722c2776ef420dcd4f8b4c28a1fc4fa70a8"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00029-of-00195.parquet": {"num_bytes": 99843928, "checksum": "59592b7d34003be2fd97e1177b24eb9ccda2381c964d31bbdb3cdf7391bb0395"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00030-of-00195.parquet": {"num_bytes": 103428341, "checksum": "4249829b8f6b78d58d058b12fd9153931161c9055851fa8fedcc25c75908b628"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00031-of-00195.parquet": {"num_bytes": 98065680, "checksum": "c06f471172543e6a80988df01664596c47a50c8c90152eb331107262d4635d8a"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00032-of-00195.parquet": {"num_bytes": 101062510, "checksum": "93e1dc952669620ffb40602d705202189a4961000c7237ab4cf8c708fa45b983"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00033-of-00195.parquet": {"num_bytes": 102833091, "checksum": "f0351163b22218158fc99fb493be7d50711dc6e625e523f084be70bc88e79a71"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00034-of-00195.parquet": {"num_bytes": 104779790, "checksum": "836fd3e6b8afb104cf6fb99ed036c8a9086a5a07a1aaec316e0637c2026b1ca7"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00035-of-00195.parquet": {"num_bytes": 99821835, "checksum": "6f22c3e74bb95a402f4fe15ac15a476811bfe84f67f32a766490fbe4a3112c0a"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00036-of-00195.parquet": {"num_bytes": 93542275, "checksum": "b65270209a6d5468c9208b53a3a65d1d53eb72782aff62d9e272d352d7123efa"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00037-of-00195.parquet": {"num_bytes": 99260833, "checksum": "1e5af547d99987f1b4a62b5ad7a0dfb48cd7813036aa1e40e43bd6c35b21ef7c"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00038-of-00195.parquet": {"num_bytes": 101903896, "checksum": "649be42de88ee9ca99793fa28c4bde271092d0fe2263eac847c5fb5a95bd1fb6"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00039-of-00195.parquet": {"num_bytes": 96184770, "checksum": "7c23f7afbea59bd90e2541975f284141fb7824fc9624e3ecca73a10037555008"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00040-of-00195.parquet": {"num_bytes": 99638693, "checksum": "894928b865d2a27f7ce8fb267a32140b4ee17dede09b7ef0275d1aa33f8374b8"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00041-of-00195.parquet": {"num_bytes": 103277487, "checksum": "86af36a8e41fc241d3d0110ceb0fe3b594e900fda8a0ae6f14d3518d5800ba00"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00042-of-00195.parquet": {"num_bytes": 101901393, "checksum": "6a0990bf99417085a7b471a06e1908b3ba1376c18b8c20da7442e82a78e80723"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00043-of-00195.parquet": {"num_bytes": 98995965, "checksum": "d4d29bc932650254f45b397163ecaf164adf0eb23224f4637a21b9897eeb13ec"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00044-of-00195.parquet": {"num_bytes": 98867988, "checksum": "a5297faf3f40de64c846ed550a0a4880a5394ed9d788a8ae447a684e88078dce"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00045-of-00195.parquet": {"num_bytes": 101367819, "checksum": "812f0f068fdf263ef81b0b7c945843eff13240ebec7044269245ceab56d6ec22"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00046-of-00195.parquet": {"num_bytes": 103235035, "checksum": "1731976ae6122151f863100c6634918882aadf0295122678407d66ef9906b5f0"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00047-of-00195.parquet": {"num_bytes": 102667277, "checksum": "1a5b40477c5a6898d5aebe6c05c377d79d22430ef6c6bc6986f45f3ba06f84ef"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00048-of-00195.parquet": {"num_bytes": 97114352, "checksum": "5e7e38d0ced261f97f0d7d89391ec87c95b2d96f86bba4d9baa282e2a8ab1c7a"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00049-of-00195.parquet": {"num_bytes": 100689272, "checksum": "da4fbfc91841fa4bd847856848f0ecfe8a8d5a7e21c3d1895166d9bd4d58e869"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00050-of-00195.parquet": {"num_bytes": 100248465, "checksum": "9847aaa601cef938e5e02b876d76f3754a7411a2a90154b07464d03f6c77bef9"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00051-of-00195.parquet": {"num_bytes": 103017674, "checksum": "542d733e3cfb5090818147d1b19f6bd18bd7a0bbec861a54c5905a108e19edde"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00052-of-00195.parquet": {"num_bytes": 103144949, "checksum": "cac8e3a0eebeada0f2228d1519ae8d89e5360e8bfd4d6b41abf1953102cdd698"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00053-of-00195.parquet": {"num_bytes": 99702224, "checksum": "f8c2d63faac987cffddc1ea8f42a7852e130b8616d0568f33c8121825619d62c"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00054-of-00195.parquet": {"num_bytes": 97196525, "checksum": "5e40c5f3df24a4cf01531a9d9fd4af19181d66433cf646d03293316284756f75"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00055-of-00195.parquet": {"num_bytes": 105572479, "checksum": "7841914ab7dbaa5a6e283fbf3bcdfe64e0a8a65048964a2f057db1cb1272074e"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00056-of-00195.parquet": {"num_bytes": 100644911, "checksum": "18ebedad80158a140b479042967711f184ab3ac1bc9687c913adf64f8e5ec6d2"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00057-of-00195.parquet": {"num_bytes": 97622217, "checksum": "49aaae5d5b073f39e3f9f053e7df2b58067b603363334d4907c342f8b63aeb13"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00058-of-00195.parquet": {"num_bytes": 98067309, "checksum": "0bcc13cc1d7c9522e1ece3881183136930a9a2a885026e9c5de102e393d50950"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00059-of-00195.parquet": {"num_bytes": 99835430, "checksum": "f01d491f86d17f5bab76eb1404125c7e4e7fbdcbc68304cd6946741888377329"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00060-of-00195.parquet": {"num_bytes": 102597239, "checksum": "04b0cdd02a4011931639f70a830174c549fb308f64afbbe164a42a51b7a3162f"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00061-of-00195.parquet": {"num_bytes": 97821295, "checksum": "30fe00a6b77ed16fb405602603ac8d3463a2ce5d44818a51a756e74012d3cb04"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00062-of-00195.parquet": {"num_bytes": 97969074, "checksum": "5a7c98355c9d2221fedbc208966661fcb52ea89f64ca761a8b201e8ceef02efc"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00063-of-00195.parquet": {"num_bytes": 98750980, "checksum": "23db78d2c19b4ac066a4b229599a30e602f5d91f7ca8c9bf38a91acf1d4cbc38"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00064-of-00195.parquet": {"num_bytes": 100061595, "checksum": "ce73da000d792def82b552c04619ae4cb8d2ec5e365b683e20b0656e70072ebc"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00065-of-00195.parquet": {"num_bytes": 98190161, "checksum": "a44c017cbdc5af0730fd9ca1559426063cad511e3270cd65c57c3e9b6f8fdfdf"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00066-of-00195.parquet": {"num_bytes": 98522006, "checksum": "436397e389b6273f9fd24f8966ce918b5c96bc8cd77e577780d0f7e8296099bc"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00067-of-00195.parquet": {"num_bytes": 102761612, "checksum": "2f597c2cc8c101b7659c6a49bad4fc5efbf29f8588c4cfc43b0f572ebbea7b11"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00068-of-00195.parquet": {"num_bytes": 100386420, "checksum": "9959133152b21a31b6d15051e3ac5600ff278c3fdcbf90dadbe1a53d348d07c4"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00069-of-00195.parquet": {"num_bytes": 100968069, "checksum": "d2d816884c306b5b29c7252f97c6958c51caeed267c33ffa1ef5c3d2989a81e2"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00070-of-00195.parquet": {"num_bytes": 99963790, "checksum": "a49bec4730dafc104c60107fd155d72c3c4d8e7a71bcca2893feee28bbffac80"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00071-of-00195.parquet": {"num_bytes": 97248974, "checksum": "a13cff42cb0617ebf6f2497265d602df24e5fd1d41b8e0f5eff83bbe45fc4731"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00072-of-00195.parquet": {"num_bytes": 104225622, "checksum": "3223b0515421003beaa3fef98504e40cfb00350f94c3725783a15a923b8e15c3"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00073-of-00195.parquet": {"num_bytes": 100460519, "checksum": "f74626d9dad6156ce95a5e671e226781e1d51bcd3a518631d37d52a9792f2b76"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00074-of-00195.parquet": {"num_bytes": 100764484, "checksum": "6d930ea8f4a71b6f3b548764e932b26e5f82056f23a4c3aac8e1a6aba2c7b447"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00075-of-00195.parquet": {"num_bytes": 109015417, "checksum": "b8d644480440592c9c680adbd7c8ffe5651fa8ecafabea5776c1b5ea4a8a7892"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00076-of-00195.parquet": {"num_bytes": 100757779, "checksum": "729edb903b77123bde14d3132dbbedb8f528c08146cae9757a4386c5fa2f3de0"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00077-of-00195.parquet": {"num_bytes": 97654748, "checksum": "3a72c5b07c1ee587e292985a9d47d61fbb6d352ef1d142fdb994238ff11bf4e2"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00078-of-00195.parquet": {"num_bytes": 100422146, "checksum": "79642171f09e1dc4bb814bb42814616518e35fde9e639a4b941cb5a842d73abc"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00079-of-00195.parquet": {"num_bytes": 99676697, "checksum": "0ff67e540713fb6d743064ec3780f99935fccf1c0fef2d40533f21aa79682d32"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00080-of-00195.parquet": {"num_bytes": 99748169, "checksum": "8fc6244da5f7fdf2229ab2053669b329bfa4bb11c02b7d06fe45380e44fd568b"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00081-of-00195.parquet": {"num_bytes": 96963878, "checksum": "5effa91f140468ae9812ca5da21c2327d5af32861b7f39ae2eb4ceab89c1d0db"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00082-of-00195.parquet": {"num_bytes": 106574443, "checksum": "a928cd202c1054965252d41bb05dd0839bc5ac7deace43ef917c805cb2ba5076"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00083-of-00195.parquet": {"num_bytes": 101682892, "checksum": "0e13b62f6eeb8aca263b6b5785f6932cf435339f6dac07ebbb87611df786de09"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00084-of-00195.parquet": {"num_bytes": 100266629, "checksum": "c44d69efb2931ac4dead58e3765971eb4c237a0fcc5f30608d42cfea220479c9"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00085-of-00195.parquet": {"num_bytes": 102561645, "checksum": "bc8aac0868179f80a03662e8b780f25d941933753d1ff43d474d335a9dce79fd"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00086-of-00195.parquet": {"num_bytes": 102408655, "checksum": "6dc141d0667aacfdbd1a05d4e771a7b7590b322a75db77ff23d8de6f904f99cc"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00087-of-00195.parquet": {"num_bytes": 100154367, "checksum": "2a42714a2215e968deef5fc3292da1e9ff95af061ddde64f9b278bd98d75fd99"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00088-of-00195.parquet": {"num_bytes": 97374168, "checksum": "7a597fc516fd7c7b3d3332d02cf0222f57c6968204ed4d5fe35666e7f5f44c87"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00089-of-00195.parquet": {"num_bytes": 98400762, "checksum": "2cb25e42789b2b2b215503132a01900ef39370cd62552f2be023179db49e0f08"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00090-of-00195.parquet": {"num_bytes": 96908400, "checksum": "414d30bea7d79bdfb2296869d195dfe1fc86ff3aa0ad9795f9156261f6e08b2a"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00091-of-00195.parquet": {"num_bytes": 104726738, "checksum": "9ca8192f71936465126338e678d48f6a8b51fd8b75d63b3f39cd207775542d47"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00092-of-00195.parquet": {"num_bytes": 98737345, "checksum": "590192cc626f2948996cf01b21334859c401c2908aaae417b8fb116b410d391e"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00093-of-00195.parquet": {"num_bytes": 97811903, "checksum": "af783e568a0cf3718153c0ef84a3dde48ef8c151e5082548fdfdfdaa02907a15"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00094-of-00195.parquet": {"num_bytes": 97845579, "checksum": "10b24292604d9593f9923cfa0f66b2465098ac63d0cdb1e2242ebe03775080ff"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00095-of-00195.parquet": {"num_bytes": 102364531, "checksum": "e83de7d41cd962b508ffe6f806c50f70d9d69205b5f350e430bce28370538562"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00096-of-00195.parquet": {"num_bytes": 103473423, "checksum": "2d68e0cf446c2d3a0254d19e4a8452aa5fe19f2db2659a92445a472defc80265"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00097-of-00195.parquet": {"num_bytes": 98971504, "checksum": "9898f4ea61fe7416c1588b03e4869f7a9f82cd27a2f8718c63cb4a6eed666e26"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00098-of-00195.parquet": {"num_bytes": 98320725, "checksum": "0d967ea4be64d58b719a7ad65b7dea33ff6d68a6fc7682af60ff229bbbe896bb"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00099-of-00195.parquet": {"num_bytes": 103256529, "checksum": "ec07aad86aeb18e6089791e068b1cce42164499ccbf0d682a54a8dad255a812a"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00100-of-00195.parquet": {"num_bytes": 96813983, "checksum": "7bda4f6b4fa5ecd240bd1ee53e8adb45dd198be689bf9625efbfdda2b6b09baf"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00101-of-00195.parquet": {"num_bytes": 100314977, "checksum": "7539a22079e47a37bd1f91d86d3726a7ba037d243c7c5da59a775144a743c01f"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00102-of-00195.parquet": {"num_bytes": 97107708, "checksum": "4dbe56b67994f119f1c2a1699555bdbf25209a8f07c7a640d93c42f68799a7f8"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00103-of-00195.parquet": {"num_bytes": 99113340, "checksum": "78a5f95f48180c4fc7074ba402c45eac4f27464042fa24bdd93e32b5d022f498"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00104-of-00195.parquet": {"num_bytes": 99668019, "checksum": "f4ec4a8bdbc089f4edc2ab03c67cbfb767c6982365851bd681766b3bb6e55a27"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00105-of-00195.parquet": {"num_bytes": 97189527, "checksum": "9f5c93ec6f95ee12c12df7fd314a1e1a3bd8b728d76212ecd9d273735aaea8e2"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00106-of-00195.parquet": {"num_bytes": 97554683, "checksum": "f19aff5e79528476c46062a24a79f913a1ee2f2e1f0ada4267e56066608f8fec"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00107-of-00195.parquet": {"num_bytes": 98059417, "checksum": "209473b229565073e36fa6bdd8aee5e2485bf17639f3af5a8e4cc65ddc02abf6"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00108-of-00195.parquet": {"num_bytes": 102041442, "checksum": "b9ffcd76dea99445e0c859ad9fc7341c07282be01b294e31bcaf305ec6428af7"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00109-of-00195.parquet": {"num_bytes": 99839921, "checksum": "6a50b733840a2706f6a109c89a58c4e66e180829586ac404150a7c1078245c35"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00110-of-00195.parquet": {"num_bytes": 98026862, "checksum": "9ba4dec5f69cf6fdea68d35c7121beb1fa834f8d8869370f1d7ce150ff79a5df"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00111-of-00195.parquet": {"num_bytes": 96283137, "checksum": "0b3934702b8fbe2113a51e8d81e80c7824c2e5a4cc9b9b57c47f37e0fcc3db90"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00112-of-00195.parquet": {"num_bytes": 100060752, "checksum": "0b4c4fee8e34c1ba7d7f9e55768a6084fc5e693f1409742955252f0b3b615ce1"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00113-of-00195.parquet": {"num_bytes": 100249833, "checksum": "1f74e547df35147a1eb56733842da268c3ddb06f03c2ce8695a9ce7a3b419c3e"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00114-of-00195.parquet": {"num_bytes": 103404463, "checksum": "71f4c28c8893a58468b03d33ab9e868b7ee2a633d68948e6fc3afa44d124116b"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00115-of-00195.parquet": {"num_bytes": 96118824, "checksum": "e562848926b94458706cc17c1cc9963b5a1013cb893eb620177a662baaef5bb7"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00116-of-00195.parquet": {"num_bytes": 98420426, "checksum": "c5a8dc61245b756e4c5f2f5c858457ffa5243dc5ad5a24d9770d40ad8d73edaa"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00117-of-00195.parquet": {"num_bytes": 93317565, "checksum": "c50c5b081fa8b4937a26d4e3039f7f90ddeec6b34a71b0c0d4e5a8ee108aa03f"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00118-of-00195.parquet": {"num_bytes": 99034327, "checksum": "b0466034b166f5695bc14c5432ca96d6dc5f2cae0cf38d169f342f71e7130106"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00119-of-00195.parquet": {"num_bytes": 99335308, "checksum": "5042f5527918a3cdf9daa7f51ed3fd8e8d87bb15f35fec8aac44b85921ce28b5"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00120-of-00195.parquet": {"num_bytes": 102895161, "checksum": "6dc31c190092a6f17f947052f820c14bb6744c4da66c2f309df98aede99285a1"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00121-of-00195.parquet": {"num_bytes": 100281363, "checksum": "28f362afb85055d31e9cb905133e757aae26b1de4827baa4ffa307830921265e"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00122-of-00195.parquet": {"num_bytes": 103444364, "checksum": "439973752159f38802925ccaa1b6324567b6d2a2059a070eaa2c1a2b5c0318e8"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00123-of-00195.parquet": {"num_bytes": 97909776, "checksum": "9a15a3808c76fc4ac98ff41d957bf73e75710d5ecf7a739bc0927c7247effe8f"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00124-of-00195.parquet": {"num_bytes": 103881788, "checksum": "0a916ee8541a8c22684e28c34477f1b79c13382ec33951b05fbe74996d3748ce"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00125-of-00195.parquet": {"num_bytes": 95400301, "checksum": "cad71da4513fe211478d97c92307f8b1e2f7363c007479d7a19cd338410aa63d"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00126-of-00195.parquet": {"num_bytes": 95561495, "checksum": "644aae5decafb6257d6223cdb7fd4bdd6a12a36eec5e60c73d7799478a33cc39"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00127-of-00195.parquet": {"num_bytes": 102054278, "checksum": "ced45acc5b845d6096b3b75ebc9ed2dd2ea904a228e295ff4378047c59fa7c12"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00128-of-00195.parquet": {"num_bytes": 97715527, "checksum": "29be96fd1fa4619eca3490af6c57f39a1e833773892558a0692deecd090124ee"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00129-of-00195.parquet": {"num_bytes": 101700671, "checksum": "1b43cde828600a913870e7a72a1316fde6034392512870034f3850b4dc26fbf8"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00130-of-00195.parquet": {"num_bytes": 105266734, "checksum": "3c1c2b656bdf65fd35808e1a963a257859d3fc388c0aafd7d5d7838723971feb"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00131-of-00195.parquet": {"num_bytes": 101136954, "checksum": "be8e7f1b3912fe9a99c7b2457a0168a12d3d558acf410516ba23758ebac503a9"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00132-of-00195.parquet": {"num_bytes": 95469215, "checksum": "5cbab3790faff63b638362ce9431e92d5e56ee561ff5c9bb040bd9d9147fd8ce"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00133-of-00195.parquet": {"num_bytes": 96956219, "checksum": "dcba9f57d438b261f3dd5a68020a0b5abd7b398d477712f2fab92b800d8f34ae"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00134-of-00195.parquet": {"num_bytes": 103740617, "checksum": "764db74defe2203731ddd34dc5e0d8356856fa9e0a1c80e3d9ce3808a26be0c1"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00135-of-00195.parquet": {"num_bytes": 103399775, "checksum": "ff1aa9f0e00597fe90caf9fb92aa33c2fa5d0c2d331caa773a3bdd67781cb265"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00136-of-00195.parquet": {"num_bytes": 98808378, "checksum": "f03c16abb0f856d1e7f577e08fd31a1be141b39ca7ba238ed26a207647019f80"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00137-of-00195.parquet": {"num_bytes": 97965072, "checksum": "adb942a50f1ee3af5c8ec7aaacd828ccd133e77361cee67296fb7cbadc7a17bb"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00138-of-00195.parquet": {"num_bytes": 96478432, "checksum": "16c3401b554ce954d4958fe772d4abede993f104c2060fef374ff0c307b404b1"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00139-of-00195.parquet": {"num_bytes": 94740084, "checksum": "3695ed4ce8d0510698a10f7559209061e80840730b987e257e43ab7994e75df3"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00140-of-00195.parquet": {"num_bytes": 96650714, "checksum": "d40538604f390db63288d6e04abba3c9a915b14b0c25c15418bc1132683f9c12"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00141-of-00195.parquet": {"num_bytes": 101867871, "checksum": "90a00090aa98784e9ae3b893a7e5f2d20ad5072a06ec90f15a0672d3134b1dd7"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00142-of-00195.parquet": {"num_bytes": 99662820, "checksum": "b2751c0b153525b6095036b7b1419148615852a4c8b7159c90db372cb33d673c"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00143-of-00195.parquet": {"num_bytes": 102238828, "checksum": "f573217b678f5a2fa2facaa722088477ef3263704205d84a2bde09adec7b871b"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00144-of-00195.parquet": {"num_bytes": 102533403, "checksum": "46f8cc449fa524c49d39ef004be6c98e851d5ec2eaa8e17c2ca0467ff0d81572"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00145-of-00195.parquet": {"num_bytes": 101040293, "checksum": "f57eb8ade6773b080083fc4ef00207ec7f9d12b7904738d01b96d9df7a7961cb"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00146-of-00195.parquet": {"num_bytes": 100308225, "checksum": "497a2fe5aec8ae4ccd9625f6640bca632913a97d693b9087d35a6f9cfda3363a"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00147-of-00195.parquet": {"num_bytes": 94805243, "checksum": "2824cd17a221b063a80b76f50bf21cb728d27e743185595a613963d57992fd9d"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00148-of-00195.parquet": {"num_bytes": 99379848, "checksum": "c0b3afd548752e89e1116d0495c344ad22f06ae93517542181320d944503ac5a"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00149-of-00195.parquet": {"num_bytes": 95601057, "checksum": "da48c875d30e403201e390f195259ab01c2f880d6c0d055abac6962367b54b65"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00150-of-00195.parquet": {"num_bytes": 98942213, "checksum": "3ad2dc2c16817a1e600da03d954a708d6cd0b40c97ca05cbbe373c426c915ba5"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00151-of-00195.parquet": {"num_bytes": 95674961, "checksum": "c308e6c026a67e4fd93670f75da83b2cfaeeb12b36270750463fcc3296960fb9"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00152-of-00195.parquet": {"num_bytes": 101542150, "checksum": "3f4056d80f77c3fa9f0f6479b1f8361a60d8c0f6f5e554d1e10e1638b4248e12"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00153-of-00195.parquet": {"num_bytes": 101690299, "checksum": "0fcd96044e2fb51bbd9ece81b84316f6508da601eaecb195c1284bbf8b7fbc2a"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00154-of-00195.parquet": {"num_bytes": 101247507, "checksum": "f4a5d8dd3371af4384e73704d6bc8e7572228baedf1313b30a721c159e64a101"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00155-of-00195.parquet": {"num_bytes": 100750517, "checksum": "527e07b87f19f495e44627d34f70610f1348df757179fbe4966030496df787aa"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00156-of-00195.parquet": {"num_bytes": 98466660, "checksum": "c572b1aa9c93c09fd482268c2e7af1f113f753b048956a20d83383fa28e859a6"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00157-of-00195.parquet": {"num_bytes": 97015771, "checksum": "eb7b8e5d63a6b8cb251f32ae612b76d41c2e10e3cac50918b848eee384505080"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00158-of-00195.parquet": {"num_bytes": 99567978, "checksum": "e611b3b9be4c820b7e21ea9ed4afb51fbc202ab75182a46c1a64229126fdb5bd"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00159-of-00195.parquet": {"num_bytes": 101337232, "checksum": "d81fcbd53c1a45b55ddb13b8cfba00589a84860561cbb4217f632c66fb9e219d"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00160-of-00195.parquet": {"num_bytes": 102349720, "checksum": "6b1a5e5c7b8980fb47a1db9a27200d147de3fe583bce49289a67168b328b773b"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00161-of-00195.parquet": {"num_bytes": 97684291, "checksum": "e07bac49991a9b2194524fbaa3c3677fa8e158dd6d92adb6153ac70759546654"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00162-of-00195.parquet": {"num_bytes": 98694984, "checksum": "8f3e83c01b384381e13ea3a2c18f5d593b4013c9f2775985936f55efe5058e64"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00163-of-00195.parquet": {"num_bytes": 100429869, "checksum": "2d1493c55c2a5954f261e09194f06f5e841ebdd77cec76da859eaabb39ea30e7"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00164-of-00195.parquet": {"num_bytes": 95622880, "checksum": "72bb66cd5cb44bd6176c5641528a01715a0e3e8505e97101cadec75efa4350b5"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00165-of-00195.parquet": {"num_bytes": 103582800, "checksum": "2188152909da0577e5df13f9ea9a725c82a907ac2a513a1ace7f9c02a40aafbd"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00166-of-00195.parquet": {"num_bytes": 99182897, "checksum": "c07cd9fe4971121349dacb98bd41f7a84869791189cbe3c345a0d160832c2acb"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00167-of-00195.parquet": {"num_bytes": 96442069, "checksum": "6ac6f3fb23ec4c11830bac10ec8cca8e6a0f46025ae342717d417f98775495e2"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00168-of-00195.parquet": {"num_bytes": 102991632, "checksum": "11279809465ddfe7aa2bc82bae0e28652c6e43bbbdd3c0457cb1f6a9f8b41b3f"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00169-of-00195.parquet": {"num_bytes": 99122587, "checksum": "834d2c024a4d24e82d0c5bf67b878bc181a122dfcf8ec3638081266e2560381f"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00170-of-00195.parquet": {"num_bytes": 102519243, "checksum": "bc4b08e173dfd74686c858c2d77c1412dba8555e7f191d90118f700e7f2a61d3"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00171-of-00195.parquet": {"num_bytes": 101920415, "checksum": "7d7e121d47227065ef74e0b1010075d513d3a0d1be8b6f88eb73423f3c739880"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00172-of-00195.parquet": {"num_bytes": 101031159, "checksum": "56793041534cd23401359424be9c77645dedddaea90324344d5388168fe1f5a1"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00173-of-00195.parquet": {"num_bytes": 97079912, "checksum": "40d0532e94a94eabcaf0fe2d2d902d38537371f372e3edcd50328598caed5ee1"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00174-of-00195.parquet": {"num_bytes": 97450447, "checksum": "a67f412750caf5d99740f8bf9160a62ce4e220d362b55faf9ac37bdd3e7db885"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00175-of-00195.parquet": {"num_bytes": 97280969, "checksum": "837779a5822c200c9faa82e73d95d9a66320e1a98d32880c57e32e7469f13cd3"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00176-of-00195.parquet": {"num_bytes": 98953073, "checksum": "33c4fd910c88989595953308b2cece88f05f6b892a97e8faf9964f7bdd57dd6f"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00177-of-00195.parquet": {"num_bytes": 100295634, "checksum": "558ddc181c2ed09d84b0559317826123547ef95311d3de18a6e6d9bc632b6f20"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00178-of-00195.parquet": {"num_bytes": 93569469, "checksum": "e555476687cb0ba04dd363679e8e1adfc3feb08531c7f1d6756ffb96f2ec8864"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00179-of-00195.parquet": {"num_bytes": 103447258, "checksum": "707d47f7553b836cdbd33f62d8196861c6a5fd54302a579d72377f29338c4a74"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00180-of-00195.parquet": {"num_bytes": 101680169, "checksum": "b79ca4bbd902873678e65d125fa2fafa9176b0e9ad3646241dc7d489f74cd416"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00181-of-00195.parquet": {"num_bytes": 102429613, "checksum": "a93866d4a4d05d2c1f5aead43589907adf060c4941fe44cdc1cb26622d5c9084"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00182-of-00195.parquet": {"num_bytes": 100985811, "checksum": "635b1a1f8f4349e652bd6b1613e6567414180b7df97795a34a0b1f3a612eef16"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00183-of-00195.parquet": {"num_bytes": 100006681, "checksum": "27b573571b4ac36eaf7a833a95f17601f0d28996e73320829072e0f492f031f9"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00184-of-00195.parquet": {"num_bytes": 100942100, "checksum": "167934d7e5e0033676a5b97abfb02eba76fffcb944b32ff7f126b5683e078906"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00185-of-00195.parquet": {"num_bytes": 95473034, "checksum": "2c2580d0812dbb3a0c6bcd20bb2c8eb4be0ef718114744d53a4111fe2ae23bd4"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00186-of-00195.parquet": {"num_bytes": 99736017, "checksum": "8bb132b7c63442b3280c79ddb0f0078d81ce390252c3d9fe58dd67eadae01e83"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00187-of-00195.parquet": {"num_bytes": 97266290, "checksum": "8660a3deccd2e7856af4f9f16528e57b0326801396ef40bb102e247dd846b1cd"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00188-of-00195.parquet": {"num_bytes": 102411030, "checksum": "0b157b6a800f01ac12f3d8c2e4db4394295ee1c522509ff55ecd0ddfbcc1ac7e"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00189-of-00195.parquet": {"num_bytes": 101124897, "checksum": "9be416392b9cf7091c447255007ed3fd7a4f8556b9d7a9c0a8d60ebd764275b8"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00190-of-00195.parquet": {"num_bytes": 99427113, "checksum": "98e5209d1e26ac173941f12848f21ba383fe0f1a9b3c9f7c387d89ec21bfa660"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00191-of-00195.parquet": {"num_bytes": 95185557, "checksum": "04944cf173eb687a15adb6250fc629ca2b439cefe6e497cafa2f6245c40a4f05"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00192-of-00195.parquet": {"num_bytes": 101191786, "checksum": "51b7394a5c6de170df474e28068cc25fd5b1b5b8c9a0f3ee36b55544d33bcf35"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00193-of-00195.parquet": {"num_bytes": 102965719, "checksum": "e2d7c0e0a429702622f0faa8b8ee755b008b3a968e82eb4f4c78d786c1e66413"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/train-00194-of-00195.parquet": {"num_bytes": 94367267, "checksum": "6a6d03411e314b7a2b695603127f6f6c1309824dd7f6550d9f0d6150fffd8067"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00000-of-00025.parquet": {"num_bytes": 99392035, "checksum": "153906c03cface6ad057d928ba05fb50b858e6cd736de7f214f077f65b081c4a"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00001-of-00025.parquet": {"num_bytes": 97236393, "checksum": "57a463464c9e4dda3a166f608db45138f1c151b26c11acdb873ce916dfef1d4b"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00002-of-00025.parquet": {"num_bytes": 102553393, "checksum": "cc237ad2a644e4d867ea48c11f1ef1800a5a8f63023b4e223a3fcba0a6eb3543"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00003-of-00025.parquet": {"num_bytes": 99099381, "checksum": "2ff2c226821ec253ffa2284fc5ecbd7bca122f09be51b1dbb8cb16e25aca16c5"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00004-of-00025.parquet": {"num_bytes": 92695260, "checksum": "6a3eceb029be0261bc4e63dc1491defae7f5ebce9acbb4902cd596c9a9ae24a8"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00005-of-00025.parquet": {"num_bytes": 95422177, "checksum": "674124d498e04244ad9067b32dc893284e2eb745a5730742f99e351e4d83e29a"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00006-of-00025.parquet": {"num_bytes": 95994458, "checksum": "4e0e8561820484bcae10cfc83b8c4c5929c414d5bbfd5d986a6a12cdafcce6a8"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00007-of-00025.parquet": {"num_bytes": 92521309, "checksum": "1fe04f94c87cfacbcc42eab7eb91009b4123dd8b6f571c86be71d01fd5d868df"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00008-of-00025.parquet": {"num_bytes": 95926731, "checksum": "5785d204828bb8c972d320ca5cb4491088c0b775157dc2fdbee9ec9114951288"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00009-of-00025.parquet": {"num_bytes": 101594507, "checksum": "585bd5d93908f924964d455420dcb1b854f1d8b1cbce0b524d1ce6852355175a"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00010-of-00025.parquet": {"num_bytes": 98693277, "checksum": "e2a4ac18cd0414c79ce9751e0470c074f103c40167da7dfd3c9b104b1fe36b8f"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00011-of-00025.parquet": {"num_bytes": 96651694, "checksum": "499fa211730035f2035fe4180ee2b2d566e8809d3ac645ab916d79e7611b1c72"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00012-of-00025.parquet": {"num_bytes": 99550139, "checksum": "7e9bbd8945d370d2a7d7ccd4021e52a11e8819c7396ee91469624b6e714b3312"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00013-of-00025.parquet": {"num_bytes": 95766815, "checksum": "9481f39574283998a3b111f25cc3b11d49fa17edb6fe204ceed41b942d8c4fd9"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00014-of-00025.parquet": {"num_bytes": 95415472, "checksum": "9b279f6b88aef440e8a7e494dc5a62eb5b35b31df201d7af0eb65c9008871b3e"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00015-of-00025.parquet": {"num_bytes": 93017301, "checksum": "9ec8a787fc64894a4293550d79b4c16780816099e250a509a1e72c2643fe17af"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00016-of-00025.parquet": {"num_bytes": 95304596, "checksum": "c2fc163609b71014ebef6a9cfbdf423be2b735d88381d55f8ebbbd6fd88bfc3b"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00017-of-00025.parquet": {"num_bytes": 100028386, "checksum": "8629fefb8ff10bd1f596195034dc3b21f255554003db5ff8e266b4fb2817e36b"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00018-of-00025.parquet": {"num_bytes": 98576761, "checksum": "d489b676fb1449b5cebc486e3ddb239d4a0d4050ded41a89b4426f58e9c994d1"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00019-of-00025.parquet": {"num_bytes": 96858053, "checksum": "e68883a5e98c43ec96ac1f288067f2d43698a31c5a37799266bf2b3f3284eb27"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00020-of-00025.parquet": {"num_bytes": 96198520, "checksum": "012d70e9d9e7da1a519e17441863d6905f1b63d19aac810d3712162543cae32d"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00021-of-00025.parquet": {"num_bytes": 96937764, "checksum": "2b401080c9deb836a6114284496471bc5cf25e369dd45caac391cb1bd15c59d0"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00022-of-00025.parquet": {"num_bytes": 95664099, "checksum": "c28e8dc738fcb97579475be3bc9811d37de85e085278e2649c27c2d2852db734"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00023-of-00025.parquet": {"num_bytes": 94201930, "checksum": "690ec735f2dbd8d21bbafa1e3099f805116fe44bc9add624a0bfd7df469c5ddd"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/test-00024-of-00025.parquet": {"num_bytes": 93893651, "checksum": "df066d37de286f6632fb6e6f368b420135a24a734f49e6cf0bf81dcdb4672638"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00000-of-00025.parquet": {"num_bytes": 98529457, "checksum": "949bb8041e9e8a72689c474b9d9921d179fff6a14ccf914498e741916c4e9630"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00001-of-00025.parquet": {"num_bytes": 97578710, "checksum": "8d862ee37c9dafc78fde7e0f6197c9d5471f531cd01628078bf8b5a53caae816"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00002-of-00025.parquet": {"num_bytes": 98986271, "checksum": "170b4d67eff76a64237f3f55c82cbd3f7065ee8f01a1d9568d7a5858fbc5bd25"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00003-of-00025.parquet": {"num_bytes": 101624201, "checksum": "2417ac064ce10bde9c0822e0cf2c38e4ab68c96e9c40b5b09c505a9c31aacd4d"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00004-of-00025.parquet": {"num_bytes": 94914949, "checksum": "090161550f2f5ae8e35a24846e2b5191441855780721603aee9fc40a7d037974"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00005-of-00025.parquet": {"num_bytes": 97203342, "checksum": "1cfbed5fd375cf52d6a4b40ea4233258b7d6fc808e3d25f4cb2b12ab5382273a"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00006-of-00025.parquet": {"num_bytes": 97732854, "checksum": "8a3b07bc2e87aac36bb73d4d5e7b483a1bc02ed2890760fdb8767d62712a03c2"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00007-of-00025.parquet": {"num_bytes": 96474873, "checksum": "d93b78ed4eddbdf13874efd47a3210185631f51093854f7820213cde02a07144"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00008-of-00025.parquet": {"num_bytes": 92754667, "checksum": "462e9e1f9bf932afc7d19759091be49c412f9630221f17595598e193a3211994"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00009-of-00025.parquet": {"num_bytes": 100032749, "checksum": "194602d267bf1882ffcabeb8705f4602bd9131618f4f3cdb4413e0b6ea78b321"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00010-of-00025.parquet": {"num_bytes": 99097158, "checksum": "f7f056b8d93e8c6aa83b2d08d2d5a4cbdf30ead4148004101dcdaed5b321fb18"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00011-of-00025.parquet": {"num_bytes": 97056747, "checksum": "9587dff56bd44103ef0a371160c39ce5f43798fc1114c7890e186d3aa1b2bb5f"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00012-of-00025.parquet": {"num_bytes": 94315352, "checksum": "aa35c78534a92d8fe5d1ca51155395d2145ad3610db11b4e881be31d056a77b1"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00013-of-00025.parquet": {"num_bytes": 96892902, "checksum": "f124bd5748823876be206f1c4d6ed22c8fe9519c43701c2162cb345ee03b85e2"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00014-of-00025.parquet": {"num_bytes": 97260890, "checksum": "28b3d6431ffb5c40ff83d0a823c1a45dd68cd32107e027af48b45066276c260b"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00015-of-00025.parquet": {"num_bytes": 95787686, "checksum": "b9d8a288be5524b93be5f4883ec8eadc10b6e14eae55328d0f0d83499edfe94a"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00016-of-00025.parquet": {"num_bytes": 95363214, "checksum": "a2060033448b52928d4b4e9a8acb7a62c05f96a92fdbe75071e19552a1c3e4f2"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00017-of-00025.parquet": {"num_bytes": 99342118, "checksum": "b4775f3c25aeca71b10fd9352e53beda9672f051fce7b3c925434c51a30a851b"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00018-of-00025.parquet": {"num_bytes": 100256356, "checksum": "641d4e1a3cf527708399fdb772ae445e597d0a5040f5313b0ab2ab58b64ead02"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00019-of-00025.parquet": {"num_bytes": 98890063, "checksum": "28ddbe3907563a77cd75fd5080f11ca37c869310b1756bfe521504466e324a88"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00020-of-00025.parquet": {"num_bytes": 98212209, "checksum": "b1c742666940cb48461ba073305c601d560067ef44337836a481a81cdc5e069c"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00021-of-00025.parquet": {"num_bytes": 101291335, "checksum": "d288c37b063a459c3b060d36ac357eaa13c3dc434450ac3a35a92d67b49cfea0"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00022-of-00025.parquet": {"num_bytes": 98278959, "checksum": "7f30c733a8a25ed2f7f9002108e5589a9d34612bad90976e6be0775fbc2af703"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00023-of-00025.parquet": {"num_bytes": 99927332, "checksum": "2b359e2095be4a06bcdb8f140681dfc2490c28cec36917aa2669d2eefa704f8c"}, "https://huggingface.co/datasets/jwh1449/AIhub_KoSpeech_dataset2/resolve/71dd923c7ed5ac671c819db3f1769960581aff29/data/valid-00024-of-00025.parquet": {"num_bytes": 98814559, "checksum": "6f8aa32b745153c08d1a611f9caf9ff8b728fad6f2278acb69dd41c56bedf5a1"}}, "download_size": 24334547387, "dataset_size": 125347837550, "size_in_bytes": 149682384937}


# def download_and_verify(url, output_path, expected_checksum):
#     if not os.path.exists(output_path):
#         print(f"Downloading {url} to {output_path}")
#         response = requests.get(url, stream=True)
#         with open(output_path, "wb") as f:
#             for chunk in response.iter_content(chunk_size=8192):
#                 f.write(chunk)
    
#     # 체크섬 확인
#     with open(output_path, "rb") as f:
#         sha256_hash = hashlib.sha256()
#         while chunk := f.read(8192):
#             sha256_hash.update(chunk)
#         computed_checksum = sha256_hash.hexdigest()
    
#     if computed_checksum != expected_checksum:
#         raise ValueError(f"Checksum mismatch for {output_path}: expected {expected_checksum}, got {computed_checksum}")
#     return output_path

# download_dir = "/home/aiuser/kospeech_parquet"
# os.makedirs(download_dir, exist_ok=True)

# data_files = {"train": [], "test": [], "valid": []}
# for url, info in dataset_info["download_checksums"].items():
#     split = "train" if "train-" in url else "test" if "test-" in url else "valid"
#     file_name = os.path.basename(url)
#     output_path = os.path.join(download_dir, file_name)
#     try:
#         download_and_verify(url, output_path, info["checksum"])
#         data_files[split].append(output_path)
#     except Exception as e:
#         print(f"Failed to download {url}: {e}")

# # 이미 다운로드된 파일 추가
# existing_files = [
#     "/home/aiuser/train-00000.parquet",
#     "/home/aiuser/test-00000.parquet",
#     "/home/aiuser/kospeech_parquet/valid-00000.parquet"
# ]
# for f in existing_files:
#     if os.path.exists(f):
#         split = "train" if "train-" in f else "test" if "test-" in f else "valid"
#         data_files[split].append(f)

# Parquet 파일 로드
try:
    dataset = load_dataset("parquet", data_files=data_files)
    print("Dataset 객체:")
    print(dataset)
    print("\nTrain 스플릿:", dataset["train"])
    print("Test 스플릿:", dataset["test"])
    print("Valid 스플릿:", dataset["valid"])
except Exception as e:
    print(f"오류: {e}")

Resolving data files:   0%|          | 0/197 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

오류: Unable to find '/home/aiuser/kospeech_parquet/valid-00000.parquet'


In [3]:
import os
import glob
from datasets import load_dataset, DatasetDict

# Parquet 파일이 있는 폴더
data_dir = "/home/aiuser/kospeech_parquet"

# 모든 .parquet 파일 목록 가져오기
parquet_files = glob.glob(os.path.join(data_dir, "*.parquet"))

# 스플릿별 파일 분류
data_files = {"train": [], "test": [], "valid": []}
for file_path in parquet_files:
    file_name = os.path.basename(file_path)
    if "train-" in file_name:
        data_files["train"].append(file_path)
    elif "test-" in file_name:
        data_files["test"].append(file_path)
    elif "valid-" in file_name:
        data_files["valid"].append(file_path)

# 각 스플릿의 파일 목록 확인
print("Train files:", len(data_files["train"]))
print("Test files:", len(data_files["test"]))
print("Valid files:", len(data_files["valid"]))

# Parquet 파일 로드
try:
    dataset = load_dataset("parquet", data_files=data_files)
    print("\nDataset 객체:")
    print(dataset)
except Exception as e:
    print(f"오류: {e}")

Train files: 195
Test files: 25
Valid files: 25


Resolving data files:   0%|          | 0/195 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/25 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/25 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/195 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/25 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/25 [00:00<?, ?it/s]


Dataset 객체:
DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 101236
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 12655
    })
    valid: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 12654
    })
})


In [4]:
# Hub로부터 전처리가 완료된 데이터셋을 로드
from datasets import load_dataset
# low_call_voices_prepreocessed = load_dataset("jwh1449/AIhub_KoSpeech_dataset2")
low_call_voices_prepreocessed = dataset
low_call_voices_prepreocessed

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 101236
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 12655
    })
    valid: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 12654
    })
})

In [5]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 인풋 데이터와 라벨 데이터의 길이가 다르며, 따라서 서로 다른 패딩 방법이 적용되어야 한다. 그러므로 두 데이터를 분리해야 한다.
        # 먼저 오디오 인풋 데이터를 간단히 토치 텐서로 반환하는 작업을 수행한다.
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Tokenize된 레이블 시퀀스를 가져온다.
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # 레이블 시퀀스에 대해 최대 길이만큼 패딩 작업을 실시한다.
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # 패딩 토큰을 -100으로 치환하여 loss 계산 과정에서 무시되도록 한다.
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # 이전 토크나이즈 과정에서 bos 토큰이 추가되었다면 bos 토큰을 잘라낸다.
        # 해당 토큰은 이후 언제든 추가할 수 있다.
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [16]:
# 훈련시킬 모델의 processor, tokenizer, feature extractor 로드
from transformers import WhisperTokenizer,  WhisperFeatureExtractor
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [10]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [12]:
import evaluate

metric = evaluate.load('cer')

In [13]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # pad_token을 -100으로 치환
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # metrics 계산 시 special token들을 빼고 계산하도록 설정
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    cer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [15]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [14]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [17]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="jwh1449/whisper_small_test",  # 원하는 리포지토리 이름을 임력한다.
    per_device_train_batch_size=16, # 기존 16
    gradient_accumulation_steps=1,  # 배치 크기가 2배 감소할 때마다 2배씩 증가
    learning_rate=1e-4, # 기존 1e-5
    warmup_steps=500, # 기존 500
    max_steps=7000,  # epoch 대신 설정, 기존 4000
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8, # 기존 8
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=500, # 평가 1000
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="cer",  # 한국어의 경우 'wer'보다는 'cer'이 더 적합할 것
    greater_is_better=False,
    push_to_hub=True,
    weight_decay=0.01, # 과적합 방지
)

In [18]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=low_call_voices_prepreocessed["train"],
    eval_dataset=low_call_voices_prepreocessed["valid"],  # or "test"
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/tmp/ipykernel_117412/2876998388.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [19]:
# torch.cuda.empty_cache()
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Cer
500,0.729500,0.714575,24.931713
1000,0.654300,0.679919,48.001186
1500,0.629100,0.611343,18.174103
2000,0.577000,0.571449,18.597967
2500,0.479200,0.517649,15.990061
3000,0.477500,0.483490,15.452819
3500,0.435800,0.454266,15.317038
4000,0.425000,0.424617,13.046095
4500,0.396100,0.401797,12.958064
5000,0.377600,0.376812,11.797134


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/home/aiuser/miniconda3/envs/test/lib/python3.9/site-packages/transformers/modeling_utils.py:3447: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 

TrainOutput(global_step=7000, training_loss=0.477365462235042, metrics={'train_runtime': 42207.0584, 'train_samples_per_second': 2.654, 'train_steps_per_second': 0.166, 'total_flos': 3.231810181103616e+19, 'train_loss': 0.477365462235042, 'epoch': 1.1061946902654867})

In [20]:
kwargs = {
    "dataset_tags": "jwh1449/AIhub_KoSpeech_dataset2",
    "dataset": "AIhub_KoSpeech_dataset2",  # a 'pretty' name for the training dataset
    "dataset_args": "config: ko, split: valid",
    "language": "ko",
    "model_name": "whisper_small_test",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}
trainer.push_to_hub(**kwargs)
processor.push_to_hub("jwh1449/whisper_small_test")
tokenizer.push_to_hub("jwh1449/whisper_small_test")

README.md:   0%|          | 0.00/2.42k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/jwh1449/whisper_small_test/commit/b3cd90f615c5a9014a8af259e2cbdbbf5945bfec', commit_message='Upload tokenizer', commit_description='', oid='b3cd90f615c5a9014a8af259e2cbdbbf5945bfec', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jwh1449/whisper_small_test', endpoint='https://huggingface.co', repo_type='model', repo_id='jwh1449/whisper_small_test'), pr_revision=None, pr_num=None)

In [ ]:
!mkdir -p /content/drive/MyDrive/whisper_checkpoint
!cp -r jwh1449/whisper_base_test/checkpoint-4000 /content/drive/MyDrive/whisper_checkpoint/


In [2]:
# 파인 튜닝한 모델을 로드
from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer

model = WhisperForConditionalGeneration.from_pretrained("jwh1449/whisper_small_test")

feature_extractor = WhisperFeatureExtractor.from_pretrained("jwh1449/whisper_small_test")
tokenizer = WhisperTokenizer.from_pretrained("jwh1449/whisper_small_test", language="Korean", task="transcribe")
processor = WhisperProcessor.from_pretrained("jwh1449/whisper_small_test")

config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.85k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/356 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [7]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [8]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="jwh1449/whisper_base_eval",  # 원하는 리포지토리 이름을 임력한다.
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # 배치 크기가 2배 감소할 때마다 2배씩 증가
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="cer",  # 한국어의 경우 'wer'보다는 'cer'이 더 적합할 것
    greater_is_better=False,
    push_to_hub=False,
    weight_decay=0.01,
)

In [9]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=low_call_voices_prepreocessed["train"],
    eval_dataset=low_call_voices_prepreocessed["test"],  # for evaluation(not validation)
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

max_steps is given, it will override any value given in num_train_epochs


In [14]:
trainer.evaluate()

{'eval_loss': 0.3282397389411926,
 'eval_model_preparation_time': 0.0025,
 'eval_cer': 10.446998252348047,
 'eval_runtime': 2174.9361,
 'eval_samples_per_second': 5.819,
 'eval_steps_per_second': 0.727}

In [15]:
import logging
import random
import torch
import librosa
import nlptutti as metrics
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from datasets import load_dataset

# 디버그 로그 활성화 (CentOS 문제 추적용)
logging.basicConfig(level=logging.DEBUG)

# 랜덤 시드 설정
random.seed(42)

# 모델과 프로세서 로드
try:
    model = WhisperForConditionalGeneration.from_pretrained("jwh1449/whisper_small_test")
    processor = WhisperProcessor.from_pretrained("jwh1449/whisper_small_test")
except Exception as e:
    print(f"Error loading model/processor: {e}")
    print("Ensure ~/.cache/huggingface/hub is cleared and network is accessible.")
    exit(1)

# generation_config 설정
model.generation_config.forced_decoder_ids = None

# 데이터셋 로드 (최대 100개 샘플 선택)
dataset = load_dataset("jwh1449/AIhub_foreign_dataset2", split="train")
selected_samples = random.sample(list(range(len(dataset))), min(100, len(dataset)))

cer_list = []

# 선택된 샘플 처리
for idx in selected_samples:
    item = dataset[idx]
    audio = item["audio"]["array"]  # 오디오 데이터
    sr = item["audio"]["sampling_rate"]  # 샘플레이트
    reference_text = item["transcription"]  # 정답 텍스트

    print(f"Processing sample {idx + 1}/{len(selected_samples)}")

    # 오디오 처리
    if sr != 16000:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        sr = 16000

    input_features = processor(audio, sampling_rate=sr, return_tensors="pt").input_features

    # 음성 인식
    with torch.no_grad():
        generated_ids = model.generate(
            input_features,
            language="ko",
            task="transcribe"
        )

    asr_text = processor.decode(generated_ids[0], skip_special_tokens=True)

    # CER 계산
    cer_result = metrics.get_cer(reference_text, asr_text)
    cer = cer_result['cer']
    cer_list.append(cer)

    print(f"CER: {cer:.4f}")
    print(f"Transcription: {asr_text}")
    print(f"Reference: {reference_text}")
    print("-" * 50)

# 평균 CER 출력
if cer_list:
    avg_cer = sum(cer_list) / len(cer_list)
    print(f"\nAverage CER: {avg_cer:.4f}")
else:
    print("No CER data available.")

print("Processing complete.")

ModuleNotFoundError: No module named 'librosa'

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import os
import json
import librosa
import torch
import random
import nlptutti as metrics

random.seed(42)

# 모델과 프로세서 로드
model = WhisperForConditionalGeneration.from_pretrained("jwh1449/whisper_base_test")
processor = WhisperProcessor.from_pretrained("jwh1449/whisper_base_test")
model.generation_config.forced_decoder_ids = None

# 변환할 음성 파일 경로 설정
audio_folder = "/content/drive/MyDrive/STTDATA/데이터모음/원본데이터"
json_folder = "/content/drive/MyDrive/STTDATA/데이터모음/정답데이터"

# 모든 WAV 파일 목록 가져오기
all_files = [f for f in os.listdir(audio_folder) if f.endswith(".wav")]
selected_files = random.sample(all_files, min(100, len(all_files)))

cer_list = []

for file_name in selected_files:
    file_path = os.path.join(audio_folder, file_name)
    json_path = os.path.join(json_folder, file_name.replace(".wav", ".json"))

    print(f"Processing: {file_name}")

    # 전체 파일 한 번에 로드 (분할 없음)
    audio, sr = librosa.load(file_path, sr=16000)
    input_features = processor(audio, sampling_rate=sr, return_tensors="pt").input_features

    with torch.no_grad():
        generated_ids = model.generate(
            input_features,
            language="ko",
            task="transcribe"
        )

    asr_text = processor.decode(generated_ids[0], skip_special_tokens=True)

    if os.path.exists(json_path):
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            reference_text = data['RecordingMetadata']['orthographic']

            cer_result = metrics.get_cer(reference_text, asr_text)
            cer = cer_result['cer']
            cer_list.append(cer)
            print(f"CER: {cer:.4f}")
    else:
        print("No reference text found.")

    print(f"Transcription: {asr_text}")
    print(f"Reference: {reference_text}")
    print("-" * 50)

if cer_list:
    print(f"\nAverage CER: {sum(cer_list)/len(cer_list):.4f}")

Processing: 00152-F-94-ZH-A-LAR027-0025203.wav
CER: 0.0526
Transcription: 담배를 피우는 사람 수가 매년 증가하고 있다.
Reference: 아 담배를 피우는 사람 수가 매년 증가하고 있다.
--------------------------------------------------
Processing: 05388-F-1-JA-A-ATQ020-1777744.wav
CER: 0.0952
Transcription: 이 시험이 끝나면 친구랑 놀라고 싶습 놀고 싶습니다.
Reference: 이 시험이 큰나면 친구란 놀라고 싶습 놀고 싶습니다.
--------------------------------------------------
Processing: 07111-F-94-ZH-A-PDT003-1592385.wav
CER: 0.8441
Transcription: 선배님, 죄송하는데 제가 단주 제주해야 하는 과제에 있어서 제 노트북 비가 들릴 수 없는 것 같아요. 왜냐하면 제가 제주해야 하는 과제는 새로운 프로젝트로 만들어야 해서 제 노트북 비가 들릴 수 있는 것 같아요. 그래서 제가 다른 과제에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로운 프로젝트에 대한 새로